In [1]:
import requests

import pandas as pd

from datetime import datetime

from pprint import pprint

from datetime import datetime, timedelta

import plotly.express as px
import plotly.colors as pc

In [2]:
lang    =  input('¿En español (es) o en inglés (en)?') 

restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

In [3]:
headers = {
    "Accept": "application/json",
    "Content_Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [4]:
def demanda_datos(lang, restaDia, input_año):
     
    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"

    endpoint = f"https://apidatos.ree.es/{lang}/datos/demanda/evolucion?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()

    datetime_lista = list()
    value_lista = list()
    percentage_lista = list()



    for value in data['included']:
        for content in value['attributes']['values']:
                
                porcentaje = content['percentage']
                valor = content['value']
                
                fecha = content ['datetime']
                fecha = pd.to_datetime(fecha)
                fecha = fecha.strftime("%d/%m/%Y")

                datetime_lista.append(fecha)
                value_lista.append(valor)
                percentage_lista.append(porcentaje)


    df_demanda = pd.DataFrame()
    df_demanda['Fecha actualización'] = datetime_lista
    df_demanda['Energia_consumida'] = value_lista

    return df_demanda

df_demanda = demanda_datos(lang, restaDia, input_año)

df_demanda


,Fecha actualización,Energia_consumida
0,16/09/2023,605057.255
1,17/09/2023,568331.891
2,18/09/2023,660518.846
3,19/09/2023,677596.790
4,20/09/2023,674288.988
5,21/09/2023,675607.185
6,22/09/2023,651037.118
7,23/09/2023,571090.314
8,24/09/2023,535176.203
9,25/09/2023,638936.275


In [5]:
df_demanda.to_csv('Demanda_evolucion.csv')

In [6]:
colores_personalizados = px.colors.qualitative.Plotly + px.colors.qualitative.Pastel + px.colors.qualitative.Set1
colores_personalizados = colores_personalizados[:30]

In [7]:
fig_demanda = px.line(data_frame = df_demanda,
        x = 'Fecha actualización',
        y = 'Energia_consumida',
        title= 'Evolución de demanda energética diaría',
        markers= True
)



fig_demanda.show()